In [2]:
import pandas_datareader.data as web
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, resample_apply
import pandas as pd
import talib as ta
import importlib
import os
import platform
import psutil
import numpy as np
import BacktestingStrategies.Optimization as opt

#https://www.sbisec.co.jp/ETGate/WPLETmgR001Control?OutSide=on&getFlg=on&burl=search_domestic&cat1=domestic&cat2=none&dir=info&file=domestic_info150818_01.html
#https://www.morningstar.co.jp/etf_foreign/search.jsp
# start = datetime.date(2021,1,1)
start = datetime.datetime.today() + relativedelta.relativedelta(years=-10)
end = datetime.date.today()
# data = web.DataReader('7203.T', 'yahoo', start, end)  #toyota
# data = web.DataReader('1321.T', 'yahoo', start, end)  #(NEXT FUNDS)日経225連動型上場投信
data = web.DataReader('1570.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均レバレッジ・インデックス連動型上場投信
# data = web.DataReader('1357.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均ダブルインバース・インデックス連動型上場投信
# data = web.DataReader('2558.T', 'yahoo', start, end)  #MAXIS 米国株式(S&P500)上場投信
# data = web.DataReader('2631.T', 'yahoo', start, end)  #MAXIS ナスダック100上場投信
# data = web.DataReader('2559.T', 'yahoo', start, end)    #MAXIS 全世界株式(オール・カントリー)上場投信

# data = web.DataReader('BTC-JPY', 'yahoo', start, end)   #Bitcoin
data = data.astype("double")
print(data)

initial = 1000_000
fname = f"html/plot-"

#position
#0 ... display "only long" result
#1 ... display both "only long" and "long and short" result
position = 1

df_returns = pd.DataFrame()

               High      Low     Open    Close      Volume  Adj Close
Date                                                                 
2012-10-15   3530.0   3440.0   3475.0   3510.0    147467.0     3510.0
2012-10-16   3615.0   3565.0   3575.0   3615.0    267218.0     3615.0
2012-10-17   3725.0   3660.0   3675.0   3700.0    430285.0     3700.0
2012-10-18   3855.0   3750.0   3750.0   3845.0   1599809.0     3845.0
2012-10-19   3870.0   3805.0   3815.0   3855.0   1052991.0     3855.0
...             ...      ...      ...      ...         ...        ...
2022-10-07  13935.0  13675.0  13700.0  13845.0  10126888.0    13845.0
2022-10-11  13435.0  13095.0  13355.0  13135.0  14341805.0    13135.0
2022-10-12  13225.0  13030.0  13090.0  13115.0   9108483.0    13115.0
2022-10-13  13135.0  12965.0  13135.0  12965.0   7377144.0    12965.0
2022-10-14  13890.0  13500.0  13565.0  13820.0  14881748.0    13820.0

[2463 rows x 6 columns]


**Methods**

In [3]:
def execute_backtest(bt):
    output = bt.run()
    print(output)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html") # 実行結果（グラフ）
    df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
    global df_returns
    df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

In [5]:
def execute_optimize(bt, opt_params):
    output = bt.optimize(**opt_params)
    print(output)
    display(output._trades)
    display(output._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")
    df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
    df_temp["_strategy"] = str(output._strategy) + "_optimize"
    global df_returns
    df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

**MACD CROSS**

In [10]:
import BacktestingStrategies.Strategy_MacdCross as btstMACD
importlib.reload(btstMACD)
importlib.reload(opt)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstMACD.MACDCross, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

#最適化
opt_params = {
    "MACDshort": range(12, 13, 1),\
    "MACDlong": range(26, 27, 1),\
    "MACDsignal": range(9, 10, 1),\
    "MACDThresholdPlus": range(0, 500, 10),\
    "MACDThresholdMinus": range(0, -500, -10),\
    "maximize":"Equity Final [$]",\
    "method": 'grid',\
    "constraint": lambda x: (x.MACDlong > x.MACDshort) & (x.MACDshort > x.MACDsignal),\
    "max_tries": 1000
}
execute_optimize(bt, opt_params)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstMACD.MACDCross_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True, #自動でポジションをクローズ(オープン)
    )
    execute_backtest(bt)

    #最適化
    execute_optimize(bt, opt_params)

Start                     2012-10-15 00:00:00
End                       2022-10-14 00:00:00
Duration                   3651 days 00:00:00
Exposure Time [%]                    8.932197
Equity Final [$]                 1276331.0065
Equity Peak [$]                   1489635.644
Return [%]                          27.633101
Buy & Hold Return [%]              293.732194
Return (Ann.) [%]                    2.527781
Volatility (Ann.) [%]               12.089071
Sharpe Ratio                         0.209096
Sortino Ratio                         0.31291
Calmar Ratio                          0.15147
Max. Drawdown [%]                  -16.688373
Avg. Drawdown [%]                   -7.219627
Max. Drawdown Duration      858 days 00:00:00
Avg. Drawdown Duration      244 days 00:00:00
# Trades                                   34
Win Rate [%]                        38.235294
Best Trade [%]                      17.293792
Worst Trade [%]                    -10.345988
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,120,168,171,8280.78800,7650.0,-75694.5600,-0.076175,2013-06-24,2013-06-27,3 days
1,106,219,222,8642.57000,9040.0,42127.5800,0.045985,2013-09-04,2013-09-09,5 days
2,101,328,330,9547.02500,9400.0,-14849.5250,-0.015400,2014-02-19,2014-02-21,2 days
3,91,500,507,10401.23250,12200.0,163687.8425,0.172938,2014-10-29,2014-11-10,12 days
4,78,715,721,14159.74550,12990.0,-91240.1490,-0.082611,2015-09-14,2015-09-25,11 days
5,78,726,733,13054.30050,13830.0,60504.5610,0.059421,2015-10-02,2015-10-14,12 days
6,90,803,808,12009.15250,11950.0,-5323.7250,-0.004926,2016-01-28,2016-02-04,7 days
7,100,820,831,10702.71750,11160.0,45728.2500,0.042726,2016-02-23,2016-03-09,15 days
8,98,857,859,11456.43000,11380.0,-7490.1400,-0.006671,2016-04-15,2016-04-19,4 days
9,113,910,912,9838.46050,9410.0,-48416.0365,-0.043550,2016-07-05,2016-07-07,2 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 991 configurations.
  output = _optimize_grid()


Start                     2012-10-15 00:00:00
End                       2022-10-14 00:00:00
Duration                   3651 days 00:00:00
Exposure Time [%]                    8.932197
Equity Final [$]                 1276331.0065
Equity Peak [$]                   1489635.644
Return [%]                          27.633101
Buy & Hold Return [%]              293.732194
Return (Ann.) [%]                    2.527781
Volatility (Ann.) [%]               12.089071
Sharpe Ratio                         0.209096
Sortino Ratio                         0.31291
Calmar Ratio                          0.15147
Max. Drawdown [%]                  -16.688373
Avg. Drawdown [%]                   -7.219627
Max. Drawdown Duration      858 days 00:00:00
Avg. Drawdown Duration      244 days 00:00:00
# Trades                                   34
Win Rate [%]                        38.235294
Best Trade [%]                      17.293792
Worst Trade [%]                    -10.345988
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,120,168,171,8280.78800,7650.0,-75694.5600,-0.076175,2013-06-24,2013-06-27,3 days
1,106,219,222,8642.57000,9040.0,42127.5800,0.045985,2013-09-04,2013-09-09,5 days
2,101,328,330,9547.02500,9400.0,-14849.5250,-0.015400,2014-02-19,2014-02-21,2 days
3,91,500,507,10401.23250,12200.0,163687.8425,0.172938,2014-10-29,2014-11-10,12 days
4,78,715,721,14159.74550,12990.0,-91240.1490,-0.082611,2015-09-14,2015-09-25,11 days
5,78,726,733,13054.30050,13830.0,60504.5610,0.059421,2015-10-02,2015-10-14,12 days
6,90,803,808,12009.15250,11950.0,-5323.7250,-0.004926,2016-01-28,2016-02-04,7 days
7,100,820,831,10702.71750,11160.0,45728.2500,0.042726,2016-02-23,2016-03-09,15 days
8,98,857,859,11456.43000,11380.0,-7490.1400,-0.006671,2016-04-15,2016-04-19,4 days
9,113,910,912,9838.46050,9410.0,-48416.0365,-0.043550,2016-07-05,2016-07-07,2 days


...

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-107,148,155,9323.61850,8500.0,88127.17950,0.088337,2013-05-27,2013-06-05,9 days
1,131,168,171,8280.78800,7650.0,-82633.22800,-0.076175,2013-06-24,2013-06-27,3 days
2,116,219,222,8642.57000,9040.0,46101.88000,0.045985,2013-09-04,2013-09-09,5 days
3,110,328,330,9547.02500,9400.0,-16172.75000,-0.015400,2014-02-19,2014-02-21,2 days
4,106,355,361,9707.81700,9550.0,-16728.60200,-0.016257,2014-03-31,2014-04-08,8 days
5,97,500,507,10401.23250,12200.0,174480.44750,0.172938,2014-10-29,2014-11-10,12 days
6,-92,520,522,12955.55100,13220.0,-24329.30800,-0.020412,2014-11-28,2014-12-02,4 days
7,-87,528,535,13373.47200,13290.0,7262.06400,0.006242,2014-12-10,2014-12-19,9 days
8,-79,587,589,14716.78950,15710.0,-78463.62950,-0.067488,2015-03-11,2015-03-13,2 days
9,-68,600,604,16030.25550,16130.0,-6782.62600,-0.006222,2015-03-30,2015-04-03,4 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 993 configurations.
  output = _optimize_grid()


Start                     2012-10-15 00:00:00
End                       2022-10-14 00:00:00
Duration                   3651 days 00:00:00
Exposure Time [%]                    12.70808
Equity Final [$]                 1444929.4035
Equity Peak [$]                   1568421.408
Return [%]                           44.49294
Buy & Hold Return [%]              293.732194
Return (Ann.) [%]                    3.837587
Volatility (Ann.) [%]               14.418241
Sharpe Ratio                         0.266162
Sortino Ratio                        0.420035
Calmar Ratio                         0.208514
Max. Drawdown [%]                  -18.404481
Avg. Drawdown [%]                   -7.050962
Max. Drawdown Duration     1008 days 00:00:00
Avg. Drawdown Duration      214 days 00:00:00
# Trades                                   48
Win Rate [%]                        39.583333
Best Trade [%]                      17.293792
Worst Trade [%]                    -10.345988
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-107,148,155,9323.61850,8500.0,88127.17950,0.088337,2013-05-27,2013-06-05,9 days
1,131,168,171,8280.78800,7650.0,-82633.22800,-0.076175,2013-06-24,2013-06-27,3 days
2,116,219,222,8642.57000,9040.0,46101.88000,0.045985,2013-09-04,2013-09-09,5 days
3,110,328,330,9547.02500,9400.0,-16172.75000,-0.015400,2014-02-19,2014-02-21,2 days
4,99,500,507,10401.23250,12200.0,178077.98250,0.172938,2014-10-29,2014-11-10,12 days
5,-93,520,522,12955.55100,13220.0,-24593.75700,-0.020412,2014-11-28,2014-12-02,4 days
6,-88,528,535,13373.47200,13290.0,7345.53600,0.006242,2014-12-10,2014-12-19,9 days
7,-74,600,604,16030.25550,16130.0,-7381.09300,-0.006222,2015-03-30,2015-04-03,4 days
8,83,715,721,14159.74550,12990.0,-97088.87650,-0.082611,2015-09-14,2015-09-25,11 days
9,83,726,733,13054.30050,13830.0,64383.05850,0.059421,2015-10-02,2015-10-14,12 days


<Strategy MACDCross_WithShortPosition(MACDshort=12,MACDlong=26,MACDsignal=9,MACDThresholdPlus=480,MACDThresholdMinus=-150)>

**EMA CROSS**

In [ ]:
import BacktestingStrategies.Strategy_EMACross as btstEMA
importlib.reload(btstEMA)
importlib.reload(opt)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstEMA.EmaCrossStrategy, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

#最適化
opt_params = {
    "EMAshort": range(10, 100, 10),\
    "EMAlong": range(10, 200, 5),\
    "maximize": "Equity Final [$]",\
    "method": 'grid',\
    "constraint":  lambda x: (x.EMAlong > x.EMAshort),\
    "max_tries":  1000
}
execute_optimize(bt, opt_params)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstEMA.EmaCrossStrategy_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True) #自動でポジションをクローズ(オープン)
    execute_backtest(bt)

    #最適化
    execute_optimize(bt, opt_params)

**SMA**

In [ ]:
import BacktestingStrategies.Strategy_SMACross as btstSMA
importlib.reload(btstSMA)
importlib.reload(opt)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstSMA.SmaCross, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

# #最適化
opt_params = {
    "SMA_short": range(10, 100, 5),\
    "SMA_long": range(10, 200, 5),\
    "maximize": opt.optim_func,\
    "method": 'grid',\
    "max_tries": 1000,\
    "constraint": lambda x: x.SMA_short < x.SMA_long}
execute_optimize(bt, opt_params)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstSMA.SmaCross_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )
    execute_backtest(bt)

    # #最適化
    execute_optimize(bt, opt_params)


**ボリンジャーバンド**

In [ ]:
import BacktestingStrategies.Strategy_BBD as btstBBD
importlib.reload(btstBBD)
importlib.reload(opt)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstBBD.BBsigma, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

# #最適化
opt_params = {
    "n": range(10, 40, 5),\
    "nu": np.arange(2.0, 3.1, 0.1).round(2).tolist(),\
    "nd": np.arange(2.0, 3.1, 0.1).round(2).tolist(),\
    "maximize": opt.optim_func,\
    "method": 'grid',\
    "max_tries": 1000}
execute_optimize(bt, opt_params)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstBBD.BBsigma_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )
    execute_backtest(bt)

    # #最適化
    execute_optimize(bt, opt_params)

**RSI**

In [ ]:
import BacktestingStrategies.Strategy_RsiOscillator as btstRsi
importlib.reload(btstRsi)
importlib.reload(opt)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstRsi.RsiOscillator, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

# #最適化
opt_params = {
    "upper_bound": range(50, 85, 5),
    "lower_bound": range(10, 45, 5),
    "rsi_window": range(10, 30, 2),\
    "maximize": opt.optim_func,\
    "method": 'grid',\
    "max_tries": 1000}
execute_optimize(bt, opt_params)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstRsi.RsiOscillator_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )
    execute_backtest(bt)

    # #最適化
    execute_optimize(bt, opt_params)

**RSI50エントリー**

In [ ]:
import BacktestingStrategies.Strategy_RsiOscillator as btstrsi50
importlib.reload(btstrsi50)
importlib.reload(opt)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstrsi50.RsiOscillator_Entry50, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstrsi50.RsiOscillator_Entry50_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )
    execute_backtest(bt)

**ドンチャンブレイクアウト**

In [ ]:
import BacktestingStrategies.Strategy_Donchian as btstdnchn
importlib.reload(btstdnchn)
importlib.reload(opt)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstdnchn.DnchnBreakout, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

# #最適化
opt_params = {
    "dnchn_long": range(10, 50, 5),
    "dnchn_short": range(10, 50, 5),
    "maximize": opt.optim_func,\
    "method": 'grid',\
    "constraint": lambda x: (x.dnchn_long > x.dnchn_short),\
    "max_tries": 1000}
execute_optimize(bt, opt_params)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstdnchn.DnchnBreakout_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )
    execute_backtest(bt)

    # #最適化
    execute_optimize(bt, opt_params)


In [ ]:
df_returns_sorted = df_returns[["_strategy", "Start", "End", "Duration", "Exposure Time [%]", "Equity Final [$]", "Equity Peak [$]", "Return [%]", "Buy & Hold Return [%]", "Return (Ann.) [%]", "Volatility (Ann.) [%]", "Sharpe Ratio", "Sortino Ratio", "Calmar Ratio", "Max. Drawdown [%]", "Avg. Drawdown [%]", "Max. Drawdown Duration", "Avg. Drawdown Duration", "# Trades", "Win Rate [%]", "Best Trade [%]", "Worst Trade [%]", "Avg. Trade [%]", "Max. Trade Duration", "Avg. Trade Duration", "Profit Factor", "Expectancy [%]", "SQN", 
]].sort_values(by = "Equity Final [$]", ascending = False)
df_returns_sorted.to_csv("OriginalStrategies.csv")